# El problema

![GIF vuelo](https://media.giphy.com/media/3o6nV8OYdUhiuKja1i/giphy.gif)

In [ ]:
import time
import random
import math

Planear un viaje para un grupo de personas que viven en distintos lugares llegando al mismo lugar siempre es un reto.

Consideremos que los miembros de una familia vienen de todas partes de Estados Unidos para encontrarse en Nueva York. Todos van a llegar el mismo día y regresarse el mismo día, y quisieran compartir transporte hacia y desde el aeropuerto.

Definimos una lista con parejas (*Nombre*, *Aeropuerto*) que indican el aeropuerto del que parte cada miembro de la familia.

In [ ]:
people = [('Seymour', 'BOS'),
          ('Franny', 'DAL'),
          ('Zooey', 'CAK'),
          ('Walt', 'MIA'),
          ('Buddy', 'ORD'),
          ('Les', 'OMA')]

In [ ]:
destination = 'LGA'

Podemos obtener información de cada aeropuerto a partir del archivo `airport-codes.txt`.

In [ ]:
def load_airports(path):
    airports = {}
    with open(path) as f:
        f.readline()
        for line in f:
            cols = line.strip().split(',')
            iata = cols[9]
            name = cols[2]
            region = cols[6]
            municipality = cols[7]
            airports[iata] = (
                name,
                region,
                municipality
            )
    return airports

In [ ]:
airports = load_airports('airport-codes.txt')

Veamos una descripción del viaje de ida de cada miembro de la familia

In [ ]:
for name, iata in people:
    print(f"\
* {name} parte de \
“{airports[iata][0]}” en {airports[iata][2]} \
y viaja hasta \
“{airports[destination][0]}” en {airports[destination][2]}\
")

* Seymour parte de “General Edward Lawrence Logan International Airport” en Boston y viaja hasta “La Guardia Airport” en New York
* Franny parte de “Dallas Love Field” en Dallas y viaja hasta “La Guardia Airport” en New York
* Zooey parte de “Akron Canton Regional Airport” en Akron y viaja hasta “La Guardia Airport” en New York
* Walt parte de “Miami International Airport” en Miami y viaja hasta “La Guardia Airport” en New York
* Buddy parte de “Chicago O'Hare International Airport” en Chicago y viaja hasta “La Guardia Airport” en New York
* Les parte de “Eppley Airfield” en Omaha y viaja hasta “La Guardia Airport” en New York


Existen muchos vuelos al día para llegar a Nueva York desde las ubicaciones de los miembros de esta familia, todos saliendo a distintos tiempos y con distinto precio.

Podemos obtener información de los vuelos disponibles a partir del archivo `schedule.txt`.

In [ ]:
def load_flights(path):
    flights = {}
    with open(path) as f:
        for line in f:
            origin, dest, t_depart, t_arrive, price = line.strip().split(',')
            flights.setdefault((origin, dest), [])
            flights[(origin, dest)].append((t_depart, t_arrive, int(price)))
    return flights

In [ ]:
flights = load_flights('schedule.txt')

Podemos consultar los posibles vuelos de ida para un miembro de la familia (en este ejemplo el primero en la lista, que es Seymour) de la siguiente forma:

In [ ]:
flights[(
    people[0][1],
    destination,
)]

[('6:17', '8:26', 89),
 ('8:04', '10:11', 95),
 ('9:45', '11:50', 172),
 ('11:16', '13:29', 83),
 ('12:34', '15:02', 109),
 ('13:40', '15:37', 138),
 ('15:27', '17:18', 151),
 ('17:11', '18:30', 108),
 ('18:34', '19:36', 136),
 ('20:17', '22:22', 102)]

Para su vuelo de regreso, simplemente cambiamos el orden del viaje:

In [ ]:
flights[(
    destination,
    people[0][1],
)]

[('6:39', '8:09', 86),
 ('8:23', '10:28', 149),
 ('9:58', '11:18', 130),
 ('10:33', '12:03', 74),
 ('12:08', '14:05', 142),
 ('13:39', '15:30', 74),
 ('15:25', '16:58', 62),
 ('17:03', '18:03', 103),
 ('18:24', '20:49', 124),
 ('19:58', '21:23', 142)]

Una desventaja de nuestra actual representación para los vuelos es la forma en que se especifican los tiempos.

Para trabajar con valores numéricos, vamos a considerar una función que calcula la cantidad de minutos que han pasado en el día para cierta hora.

In [ ]:
def get_minutes(t):
    x = time.strptime(t, '%H:%M')
    h = x.tm_hour
    m = x.tm_min
    return 60 * h + m

In [ ]:
get_minutes('00:00')

0

In [ ]:
get_minutes('00:51')

51

In [ ]:
get_minutes('01:12')

72

In [ ]:
list(
    map(
        lambda x: (
            get_minutes(x[0]),
            get_minutes(x[1]),
            x[2],
        ),
        flights[(
            people[0][1],
            destination
        )]
    )
)

[(377, 506, 89),
 (484, 611, 95),
 (585, 710, 172),
 (676, 809, 83),
 (754, 902, 109),
 (820, 937, 138),
 (927, 1038, 151),
 (1031, 1110, 108),
 (1114, 1176, 136),
 (1217, 1342, 102)]

Vamos a considerar una estructura particular para las posibles soluciones al problema. Una representación usual es que las soluciones sean listas de números. En nuestro caso, cada número puede representar el vuelo de ida o el vuelo de regreso, de tal manera que el tamaño de la solución es dos veces más que la cantidad de personas.

Por ejemplo,
```
[1,4,3,2,7,3,6,3,2,4,5,3]
```

Nos representa una solución donde:
- la persona con índice `0` toma el vuelo de ida con índice `1` y el vuelo de regreso con índice `4`,
- la persona con índice `1` toma el vuelo de ida con índice `3` y el vuelo de regreso con índice `2`,
- la persona con índice `3` toma el vuelo de ida con índice `7` y el vuelo de regreso con índice `3`,
- etc.

El índice de las personas hace referencia a la lista `people`, si dicha persona vive en `A` y va a `B`, entonces el índice de ida hace referencia a la lista `flights[(A,B)]`, mientras que el índice de regreso hace referencia a la lista `flights[(B,A)]`.

Todo esto puede sonar confuso, lo conveniente es elegir una representación lo suficientemente simple para que nuestros programas encuentren buenas soluciones, pero lo suficientemente complejo como para entender soluciones como personas.

La siguiente función nos permite tomar un valor de solución e imprimir la información de forma legible.

In [ ]:
def print_schedule(s):
    for i in range(len(s) // 2):
        name = people[i][0]
        origin = people[i][1]
        out = flights[(origin,destination)][s[2*i]]
        ret = flights[(destination,origin)][s[2*i+1]]
        print(
            name,
            airports[people[i][1]][2],
            out[0],out[1],out[2],
            ret[0],ret[1],ret[2],
        )

In [ ]:
print_schedule([1,4,3,2,7,3,6,3,2,4,5,3])

Seymour Boston 8:04 10:11 95 12:08 14:05 142
Franny Dallas 10:30 14:57 290 9:49 13:51 229
Zooey Akron 17:08 19:08 262 10:32 13:16 139
Walt Miami 15:34 18:11 326 11:08 14:38 262
Buddy Chicago 9:42 11:32 169 12:08 14:47 231
Les Omaha 13:37 15:08 250 11:07 13:24 171


Hasta ahora, hemos modelado este problema de forma computacional y tenemos codificaciones prácticas para razonar sobre personas, vuelos y tiempos.
Sin embargo, podemos ver que la solución de ejemplo nos dice que Walt viaja desde Miami en el vuelo que parte a las 15:34 y regresa el mismo dia a las 11:08...

Esta es una pésima solución... ¡es imposible!

Para poder avanzar, necesitamos un criterio que nos permita comparar dos soluciones, claramente la que del ejemplo es pésima, pero ¿De qué forma podemos especificar que una solución es mejor que esta?

# Función de costo

![GIF costoso](https://media.giphy.com/media/yIxNOXEMpqkqA/giphy.gif)

Vamos a definir una *función de costo*, de tal manera que el problema se va a reducir a encontrar un conjunto de entradas (vuelos en este caso) que minimice la función de costo, es decir, la que tenga el costo más bajo.

Esta función de costo debe recibir una posible solución y darnos un valor numérico que nos indique qué tan mala es.

Suele ser dificil determinar qué hace que una solución sea buena o mala cuando se involucran varios factores. Consideremos algunos candidatos para este problema:

- Precio: El precio total de todos los vuelos de avión, o posiblemente un promedio ponderado que tome en cuenta la situación financiera de los familiares.
- Tiempo de viaje: El tiempo total de viaje para todos los familiares.
- Tiempo de espera: El tiempo que van a esperar en el aeropuerto a que lleguen los demás miembros de la familia.
- Tiempo de salida: Los vuelos que salen demasiado temprano pueden imponer costo adicional a los pasajeros que no durmieron plenamente en la noche por llegar al aeropuerto.
- Periodo de renta de carro: Si la familia renta un carro, deben regresarlo lo suficientemente temprano en el día para que no les cobren un día adicional.

![GIF cansado](https://media.giphy.com/media/AgP9GXdpREaURYbxca/giphy.gif)

Podemos imaginarnos otros factores que tomar en cuenta para nuestro problema particular.

Una vez que determinamos el conjunto de factores que afectan nuestra noción de *costo*, debemos determinar cómo combinarlas en un número.

In [ ]:
flights

{('LGA', 'OMA'): [('6:19', '8:13', 239),
  ('8:04', '10:59', 136),
  ('9:31', '11:43', 210),
  ('11:07', '13:24', 171),
  ('12:31', '14:02', 234),
  ('14:05', '15:47', 226),
  ('15:07', '17:21', 129),
  ('16:35', '18:56', 144),
  ('18:25', '20:34', 205),
  ('20:05', '21:44', 172)],
 ('OMA', 'LGA'): [('6:11', '8:31', 249),
  ('7:39', '10:24', 219),
  ('9:15', '12:03', 99),
  ('11:08', '13:07', 175),
  ('12:18', '14:56', 172),
  ('13:37', '15:08', 250),
  ('15:03', '16:42', 135),
  ('16:51', '19:09', 147),
  ('18:12', '20:17', 242),
  ('20:05', '22:06', 261)],
 ('LGA', 'ORD'): [('6:03', '8:43', 219),
  ('7:50', '10:08', 164),
  ('9:11', '10:42', 172),
  ('10:33', '13:11', 132),
  ('12:08', '14:47', 231),
  ('14:19', '17:09', 190),
  ('15:04', '17:23', 189),
  ('17:06', '20:00', 95),
  ('18:33', '20:22', 143),
  ('19:32', '21:25', 160)],
 ('ORD', 'LGA'): [('6:05', '8:32', 174),
  ('8:25', '10:34', 157),
  ('9:42', '11:32', 169),
  ('11:01', '12:39', 260),
  ('12:44', '14:17', 134),
  ('14

In [ ]:
def schedule_cost(s):
    # contamos el precio total de cada vuelo (ida y regreso)
    total_price = 0
    
    # nos interesa conocer el tiempo de llegada a NY mas tarde
    # y el tiempo de salida de NY mas temprano.
    latest_arrival = 0
    earliest_departure = 24 * 60
    
    for i in range(len(s) // 2):
        origin = people[i][1]
        out_flight = flights[(origin, destination)][s[2*i]]
        ret_flight = flights[(destination, origin)][s[2*i+1]]
        
        total_price += out_flight[2] # vuelo de ida
        total_price += ret_flight[2] # vuelo de regreso
        
        # tiempo de llegada máximo
        # tiempo de salida mínimo
        if latest_arrival < get_minutes(out_flight[1]):
            latest_arrival = get_minutes(out_flight[1])
        if earliest_departure > get_minutes(ret_flight[0]):
            earliest_departure = get_minutes(ret_flight[0])
    
    # contamos el tiempo de espera de cada persona
    total_wait = 0
    
    for i in range(len(s) // 2):
        origin = people[i][1]
        out_flight = flights[(origin, destination)][s[2*i]]
        ret_flight = flights[(destination, origin)][s[2*i+1]]
        
        # todos esperan al último familiar en llegar
        total_wait += latest_arrival - get_minutes(out_flight[1])
        
        # todos llegan al aeropuerto al mismo tiempo y esperan su vuelo
        total_wait += get_minutes(ret_flight[0]) - earliest_departure
        
        # si el último en llegar a NY llega después del primero en
        # irse de NY se paga un día más de la renta del carro.
        # el costo de la renta por un día es independiente de la
        # solución.
        if latest_arrival > earliest_departure:
            total_price += 50
    
    # El costo total es el precio total de los vuelos y el tiempo de
    # espera total de las personas.
    # Buscamos soluciones con un bajo costo.
    return total_price + total_wait 

Veamos qué tan malo es nuestro ejemplo con estos criterios de costo:

In [ ]:
schedule_cost([1,4,3,2,7,3,6,3,2,4,5,3])

4615

# Buscar todas las soluciones

¡Perfecto! Ahora solo tenemos que considerar todas las combinaciones de vuelos para determinar cuál tiene el menor costo.

In [ ]:
print(f"Hay {len(people)} personas en la familia:")
total_solutions = 1
for p in people:
    name = p[0]
    origin = airports[p[1]][2]
    outs = len(flights[(p[1],destination)])
    rets = len(flights[(destination,p[1])])
    total_solutions *= outs
    total_solutions *= rets
    print(f"- {name} parte de {origin}, tiene {outs} opciones de salida y {rets} opciones de regreso;")
print(f"Por lo que hay un total de {total_solutions} posibles soluciones que analizar!\n\n")

if total_solutions > 1e9:
    print("¡Caracoles! esas son muchas soluciones")
else:
    print("Está facilito encontrar la mejor")

Hay 6 personas en la familia:
- Seymour parte de Boston, tiene 10 opciones de salida y 10 opciones de regreso;
- Franny parte de Dallas, tiene 10 opciones de salida y 10 opciones de regreso;
- Zooey parte de Akron, tiene 10 opciones de salida y 10 opciones de regreso;
- Walt parte de Miami, tiene 10 opciones de salida y 10 opciones de regreso;
- Buddy parte de Chicago, tiene 10 opciones de salida y 10 opciones de regreso;
- Les parte de Omaha, tiene 10 opciones de salida y 10 opciones de regreso;
Por lo que hay un total de 1000000000000 posibles soluciones que analizar!


¡Caracoles! esas son muchas soluciones


![GIF dolor](https://media.giphy.com/media/7T33BLlB7NQrjozoRB/giphy.gif)

# Buscar aleatoriamente

Veamos si podemos encontrar un buen resultado haciendo una búsqueda aleatoria en el espacio de soluciones.

![GIF random](https://media.giphy.com/media/89Eko49m84Ja/giphy.gif)

Consideremos la función `solve_randomly` que toma dos parámetros:
1. El *dominio* que consiste en una secuencia de tuplas `(min, max)` que establecen el valor mínimo y máximo que pueden tomar las entradas de las soluciones (de esta forma, codificamos el espacio de posibles soluciones de manera sucinta)
2. Una función de *costo*, que toma una posible solución y nos regresa un valor numérico que queremos minimizar.

Es importante observar que la cantidad de elementos en el dominio es igual a la cantidad de elementos en una solución.

Vamos a generar de forma aleatoria soluciones y regresar aquella con el costo mas pequeño.

In [ ]:
def random_solution(domain):
    return [
        random.randint(r[0], r[1])
        for r in domain
    ]

In [ ]:
def solve_randomly(domain, cost_of, repeats = 10000):
    best_cost = float('inf')
    best_sol = None
    
    for _ in range(repeats):
        s = random_solution(domain)
        c = cost_of(s)
        if c < best_cost:
            best_cost = c
            best_sol = s
    
    return s

In [ ]:
domain = [(0,9)] * len(people) * 2

In [ ]:
def test_randomly(repeats = 10000):
    s = solve_randomly(
        domain,
        schedule_cost,
        repeats
    )
    print_schedule(s)
    print(f"\nCon costo {schedule_cost(s)}")

In [ ]:
test_randomly()

Seymour Boston 15:27 17:18 151 19:58 21:23 142
Franny Dallas 18:26 21:29 464 14:20 17:32 332
Zooey Akron 20:30 23:11 114 16:33 18:15 253
Walt Miami 6:25 9:30 335 8:23 11:07 143
Buddy Chicago 12:44 14:17 134 9:11 10:42 172
Les Omaha 11:08 13:07 175 12:31 14:02 234

Con costo 6931


In [ ]:
%timeit -n 1 -r 1 test_randomly()

Seymour Boston 6:17 8:26 89 17:03 18:03 103
Franny Dallas 13:54 18:02 294 12:20 16:34 500
Zooey Akron 18:35 20:28 204 12:01 13:41 267
Walt Miami 15:34 18:11 326 12:37 15:05 170
Buddy Chicago 19:50 22:24 269 7:50 10:08 164
Les Omaha 7:39 10:24 219 14:05 15:47 226

Con costo 6786
3.07 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%timeit -n 1 -r 1 test_randomly(int(1e5))

Seymour Boston 9:45 11:50 172 15:25 16:58 62
Franny Dallas 18:26 21:29 464 15:49 20:10 497
Zooey Akron 13:40 15:38 137 9:58 12:56 249
Walt Miami 14:01 17:24 338 11:08 14:38 262
Buddy Chicago 19:50 22:24 269 17:06 20:00 95
Les Omaha 15:03 16:42 135 20:05 21:44 172

Con costo 6402
29.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%timeit -n 1 -r 1 test_randomly(int(2e5))

Seymour Boston 20:17 22:22 102 17:03 18:03 103
Franny Dallas 20:07 23:27 473 10:51 14:16 256
Zooey Akron 13:40 15:38 137 16:33 18:15 253
Walt Miami 14:01 17:24 338 11:08 14:38 262
Buddy Chicago 11:01 12:39 260 7:50 10:08 164
Les Omaha 20:05 22:06 261 6:19 8:13 239

Con costo 6414
1min ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Pensando en nuestros alrededores


Intentar obtener un buen resultado generando soluciones aleatorias es una ineficiente y pésima estrategia en este caso.

Un problema obvio de el método es que no aprovecha la información de las mejores soluciones que ha generado para generar otras buenas soluciones.

En nuestro problema particular, una solución con bajo costo es probablemente similar a otras soluciones con bajo costo.

Vamos a incorporar esta idea implementando en Python un método alternativo llamado *descenso de colinas*. Comenzamos con una solución aleatoria y buscamos en la *vecindad* de la solución por aquellas que mejoran el costo.

![GIF sisifo](https://media.giphy.com/media/xT0BKumCMrUb0dCypa/giphy.gif)

Detendremos la búsqueda hasta llegar a una solución cuya vecindad no mejora el costo.

In [ ]:
def neighbors_of(s, domain):
    neighbors = []
    for i in range(len(domain)):
        if s[i] > domain[i][0]:
            neighbors.append(s[0:i] + [s[i] - 1] + s[i+1:])
        if s[i] < domain[i][1]:
            neighbors.append(s[0:i] + [s[i] + 1] + s[i+1:])
    return neighbors

Este criterio de vecindad corresponde a todas las soluciones que están a distancia 1 (de acuerdo a Hamming).

Veamos la lista de vecinos de nuestra solución de ejemplo:

In [ ]:
neighbors_of([1,4,3,2,7,3,6,3,2,4,5,3], domain)

[[0, 4, 3, 2, 7, 3, 6, 3, 2, 4, 5, 3],
 [2, 4, 3, 2, 7, 3, 6, 3, 2, 4, 5, 3],
 [1, 3, 3, 2, 7, 3, 6, 3, 2, 4, 5, 3],
 [1, 5, 3, 2, 7, 3, 6, 3, 2, 4, 5, 3],
 [1, 4, 2, 2, 7, 3, 6, 3, 2, 4, 5, 3],
 [1, 4, 4, 2, 7, 3, 6, 3, 2, 4, 5, 3],
 [1, 4, 3, 1, 7, 3, 6, 3, 2, 4, 5, 3],
 [1, 4, 3, 3, 7, 3, 6, 3, 2, 4, 5, 3],
 [1, 4, 3, 2, 6, 3, 6, 3, 2, 4, 5, 3],
 [1, 4, 3, 2, 8, 3, 6, 3, 2, 4, 5, 3],
 [1, 4, 3, 2, 7, 2, 6, 3, 2, 4, 5, 3],
 [1, 4, 3, 2, 7, 4, 6, 3, 2, 4, 5, 3],
 [1, 4, 3, 2, 7, 3, 5, 3, 2, 4, 5, 3],
 [1, 4, 3, 2, 7, 3, 7, 3, 2, 4, 5, 3],
 [1, 4, 3, 2, 7, 3, 6, 2, 2, 4, 5, 3],
 [1, 4, 3, 2, 7, 3, 6, 4, 2, 4, 5, 3],
 [1, 4, 3, 2, 7, 3, 6, 3, 1, 4, 5, 3],
 [1, 4, 3, 2, 7, 3, 6, 3, 3, 4, 5, 3],
 [1, 4, 3, 2, 7, 3, 6, 3, 2, 3, 5, 3],
 [1, 4, 3, 2, 7, 3, 6, 3, 2, 5, 5, 3],
 [1, 4, 3, 2, 7, 3, 6, 3, 2, 4, 4, 3],
 [1, 4, 3, 2, 7, 3, 6, 3, 2, 4, 6, 3],
 [1, 4, 3, 2, 7, 3, 6, 3, 2, 4, 5, 2],
 [1, 4, 3, 2, 7, 3, 6, 3, 2, 4, 5, 4]]

In [ ]:
def solve_hillclimbing(domain, cost_of):
    s = random_solution(domain)
    
    while True:
        neighbors = neighbors_of(s, domain)
        cost = cost_of(s)
        best_neighbor = min(neighbors, key=cost_of)
        neighbor_cost = cost_of(best_neighbor)
        
        if cost < neighbor_cost:
            return s
        
        s = best_neighbor

In [ ]:
def test_hillclimbing():
    s = solve_hillclimbing(
        domain,
        schedule_cost,
    )
    print_schedule(s)
    print(f"\nCon costo {schedule_cost(s)}")

In [ ]:
%timeit -n 1 -r 1 test_hillclimbing()

Seymour Boston 17:11 18:30 108 15:25 16:58 62
Franny Dallas 10:30 14:57 290 14:20 17:32 332
Zooey Akron 17:08 19:08 262 15:50 18:45 243
Walt Miami 11:28 14:40 248 15:23 18:49 150
Buddy Chicago 9:42 11:32 169 14:19 17:09 190
Les Omaha 16:51 19:09 147 15:07 17:21 129

Con costo 3648
173 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
test_hillclimbing()

Seymour Boston 17:11 18:30 108 13:39 15:30 74
Franny Dallas 10:30 14:57 290 14:20 17:32 332
Zooey Akron 15:23 17:25 232 13:37 15:33 142
Walt Miami 11:28 14:40 248 15:23 18:49 150
Buddy Chicago 15:58 18:40 173 14:19 17:09 190
Les Omaha 12:18 14:56 172 15:07 17:21 129

Con costo 3325


# Calentando los motores

Un problema del método de descenso de colinas es que una vez que se llega a un mínimo local, se deja de buscar. Sin embargo, este mínimo local puede corresponder a una pésima solución.

Podemos perturbar la búsqueda con un método llamado *recocido simulado*.

![GIF forjar](https://media.giphy.com/media/NpILbqtmLO1Qkfvc4f/giphy.gif)

Este método consiste en partir de un estado caliente, en donde es probable seguir un vecino con peor costo. Poco a poco, la temperatura del sistema baja, de tal forma que cada vez es menos probable elegir un peor vecino. Eventualmente, la temperatura es suficientemente baja como para detener la búsqueda.

Esto nos permite introducir una probabilidad variable para evitar algunos mínimos locales.

La estrategia es la siguiente: comenzamos con una solución aleatoria y una temperatura alta, tomamos un vecino aleatorio y analizamos los casos:
1. Si el vecino tiene menor costo $c'$ que la solución actual con costo $c$, elegimos al vecino como nueva solución
2. Si no es el caso, elegimos al vecino como nueva solución con probabilidad $$\mathrm{e}^{(c-c')/T}$$

In [ ]:
def solve_annealing(domain, cost_of, Ti=100000.0, Tf=0.1, alpha=0.95):
    solution = random_solution(domain)
    cost = cost_of(solution)
    T = Ti
    while T > Tf:
        neighbor = random.choice(neighbors_of(solution, domain))
        neighbor_cost = cost_of(neighbor)
        diff = cost - neighbor_cost
        if diff > 0 or random.random() < (math.exp(diff / T)):
            solution = neighbor
            cost = neighbor_cost
        T = alpha*T
    
    return solution

In [ ]:
def test_annealing(Ti=1000000.0, Tf=0.1, alpha=0.95, beta=0):
    s = solve_annealing(
        domain,
        schedule_cost,
        Ti, Tf, alpha,
    )
    print_schedule(s)
    print(f"\nCon costo {schedule_cost(s)}")

In [ ]:
%timeit -n 1 -r 1 test_annealing(Ti=100000.0, Tf=0.1, alpha=0.95)

Seymour Boston 9:45 11:50 172 10:33 12:03 74
Franny Dallas 6:12 10:22 230 10:51 14:16 256
Zooey Akron 8:27 10:45 139 10:32 13:16 139
Walt Miami 9:15 12:29 225 12:37 15:05 170
Buddy Chicago 9:42 11:32 169 10:33 13:11 132
Les Omaha 9:15 12:03 99 11:07 13:24 171

Con costo 2540
101 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%timeit -n 1 -r 1 test_annealing(Ti=100000.0, Tf=0.1, alpha=0.99)

Seymour Boston 17:11 18:30 108 8:23 10:28 149
Franny Dallas 13:54 18:02 294 9:49 13:51 229
Zooey Akron 17:08 19:08 262 8:19 11:16 122
Walt Miami 15:34 18:11 326 8:23 11:07 143
Buddy Chicago 15:58 18:40 173 7:50 10:08 164
Les Omaha 16:51 19:09 147 8:04 10:59 136

Con costo 2705
429 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%timeit -n 1 -r 1 test_annealing(Ti=1000000.0, Tf=0.1, alpha=0.99)

Seymour Boston 9:45 11:50 172 12:08 14:05 142
Franny Dallas 9:08 12:12 364 17:14 20:59 277
Zooey Akron 8:27 10:45 139 13:37 15:33 142
Walt Miami 9:15 12:29 225 12:37 15:05 170
Buddy Chicago 9:42 11:32 169 12:08 14:47 231
Les Omaha 9:15 12:03 99 12:31 14:02 234

Con costo 3084
547 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%timeit -n 1 -r 1 test_annealing(Ti=10000000.0, Tf=0.1, alpha=0.99)

Seymour Boston 12:34 15:02 109 10:33 12:03 74
Franny Dallas 10:30 14:57 290 17:14 20:59 277
Zooey Akron 8:27 10:45 139 13:37 15:33 142
Walt Miami 11:28 14:40 248 12:37 15:05 170
Buddy Chicago 12:44 14:17 134 10:33 13:11 132
Les Omaha 9:15 12:03 99 11:07 13:24 171

Con costo 3266
554 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%timeit -n 1 -r 1 test_annealing(Ti=10000000.0, Tf=0.1, alpha=0.999)

Seymour Boston 8:04 10:11 95 10:33 12:03 74
Franny Dallas 6:12 10:22 230 10:51 14:16 256
Zooey Akron 8:27 10:45 139 10:32 13:16 139
Walt Miami 7:34 9:40 324 12:37 15:05 170
Buddy Chicago 8:25 10:34 157 10:33 13:11 132
Les Omaha 7:39 10:24 219 11:07 13:24 171

Con costo 2471
5.62 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Evolución al rescate?

![GIF darwin](https://media.giphy.com/media/VFAke5Xm1TDwjgimyW/giphy.gif)

Ahora vamos a considerar otro método, que al igual que el recocido simulado está inspirado en la naturaleza, pero no en la física, si no en la biología.

Primero se crea un conjunto de soluciones aleatorias, conocidas como *la población*. En cada paso del método, la función de costo para cada solución en la población es calculada, esto nos permite ordenar las soluciones de mejor a peor.

Posteriormente, una nueva población es generada a partir de la actual: Las mejores soluciones de la actual se conservan tal cuál (*elitismo*). El resto de la población, va a ser modificada para obtener la nueva población.

Hay dos formas en que las soluciones pueden ser modificadas. La mas simple es llamada *mutación* y consiste en un pequeño y simple cambio aleatorio a la solución actual. Esto sería similar a elegir un vecino de forma aleatoria.

Otra manera de modificar las soluciones es llamado *cruza*, consiste en tomar dos soluciones de las mejores y combinarlas de alguna manera. Una forma simple de combinarlas es tomar una cantidad aleatoria de elementos de una buena solución y acompletar los elementos que faltan con otra buena solución.

Una vez que se obtiene la nueva población, el proceso continúa.

In [ ]:
def mutate(s, domain):
    return random.choice(neighbors_of(s, domain))

In [ ]:
def crossover(s1, s2):
    i = random.randint(1, len(s1)-2)
    return s1[0:i] + s2[i:]

In [ ]:
def solve_evolving(domain, cost_of, pop_size=50, mut_prob=0.2, elite=0.2, epochs=1000):
    pop = [random_solution(domain) for _ in range(pop_size)]
    top_elite = int(elite * pop_size)
    
    for epoch in range(epochs):
        pop.sort(key=cost_of)
        best = pop[0:top_elite]
        while len(best) < pop_size:
            if random.random() < mut_prob:
                best.append(mutate(
                    best[random.randint(0, top_elite-1)],
                    domain
                ))
            else:
                best.append(crossover(
                    best[random.randint(0, top_elite-1)],
                    best[random.randint(0, top_elite-1)],
                ))
        pop = best
    pop.sort(key=cost_of)
    return pop[0]

In [ ]:
def test_evolving(pop_size=50, mut_prob=0.2, elite=0.2, epochs=1000):
    s = solve_evolving(
        domain,
        schedule_cost,
        pop_size, mut_prob,
        elite, epochs,
    )
    print_schedule(s)
    print(f"\nCon costo {schedule_cost(s)}")

In [ ]:
%timeit -r 1 -n 1 test_evolving(50, 0.2, 0.2, 1000)

Seymour Boston 12:34 15:02 109 10:33 12:03 74
Franny Dallas 10:30 14:57 290 10:51 14:16 256
Zooey Akron 8:27 10:45 139 10:32 13:16 139
Walt Miami 11:28 14:40 248 15:23 18:49 150
Buddy Chicago 12:44 14:17 134 10:33 13:11 132
Les Omaha 9:15 12:03 99 11:07 13:24 171

Con costo 2826
14.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%timeit -r 1 -n 1 test_evolving(50, 0.2, 0.2, 2000)

Seymour Boston 12:34 15:02 109 13:39 15:30 74
Franny Dallas 6:12 10:22 230 14:20 17:32 332
Zooey Akron 12:08 14:59 149 13:37 15:33 142
Walt Miami 11:28 14:40 248 12:37 15:05 170
Buddy Chicago 12:44 14:17 134 14:19 17:09 190
Les Omaha 12:18 14:56 172 12:31 14:02 234

Con costo 2927
28.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%timeit -r 1 -n 1 test_evolving(20, 0.2, 0.2, 1000)

Seymour Boston 15:27 17:18 151 10:33 12:03 74
Franny Dallas 6:12 10:22 230 10:51 14:16 256
Zooey Akron 15:23 17:25 232 10:32 13:16 139
Walt Miami 14:01 17:24 338 12:37 15:05 170
Buddy Chicago 14:22 16:32 126 10:33 13:11 132
Les Omaha 12:18 14:56 172 11:07 13:24 171

Con costo 3035
5.62 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# La tarea

1. Explora los parámetros y la función costo de este problema, intenta encontrar mejores valores para resolver el problema.
2. Elige un problema distinto, y modela las posibles soluciones del problema para utilizar los métodos discutidos en esta libreta: **búsqueda de solución aleatoria, búsqueda de solución por descenso de colinas, búsqueda de solución por recocido simulado, búsqueda de solución por algoritmos genéticos.**
3. Compara los resultados de las soluciones de los cuatro métodos anteriores.
4. ¿Es posible resolver el problema que planteas analizando todas las posibles soluciones? Justifica tu respuesta.